# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for **cosmetologist violations** for people with the last name **Nguyen**.

In [2]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

In [3]:
select = Select(driver.find_element_by_id('pht_status'))
select.select_by_visible_text('Cosmetologists')


text_input = driver.find_element_by_id("pht_lnm")
text_input.send_keys('Nguyen')

driver.find_element_by_name("B1").click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [4]:
rows = driver.find_elements_by_tag_name("tr")
for row in rows[:10]:
    print(row.text.strip())

Name and Location Order Basis for Order
NGUYEN, MIMI PHAM
City: KATY
County: HARRIS
Zip Code: 77449


License #: 784210

Complaint # COS20190010072 Date: 11/12/2020

Respondent is assessed an administrative penalty in the amount of $1,125. Respondent failed properly clean and sanitize the metal implements used at the Salon; Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution.
NGUYEN, HA
City: ARLINGTON
County: TARRANT
Zip Code: 76017


License #: 764888

Complaint # COS20190016762 Date: 11/12/2020

Respondent is assessed an administrative penalty in the amount of $2,250. Respondent failed to clean and sanitize four (4) whirlpool foot spas as required at the end of each day, constituting two (2) violations; Respondent failed to keep a record of the date and time of four (4) foot spas daily or bi-weekly cleaning and if the foot spas were not used, constituting two (2) violations.
NGUYEN, THAO HONG
City: SAN ANTONIO
County: BEXAR
Zip

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   # Instead of stopping on an error, it'll jump down here instead
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement. Most people use `pass`, but it's also nice to print out debug statements so you know when/where it's running into errors.

**Why doesn't the first one have a name?**

In [5]:
for row in rows[:10]:
    try:
        first_name = row.find_element_by_class_name("results_text")
        print(first_name.text.strip())
    except:
        pass
        

NGUYEN, MIMI PHAM
NGUYEN, HA
NGUYEN, THAO HONG
NGUYEN, CINDY
NGUYEN, CHAU KHANH LINH
NGUYEN, TRANG T
NGUYEN, DUNG MINH
NGUYEN, YEN NHI THI
NGUYEN, JOHNNY DAT


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...*

In [6]:
for row in rows[:10]:
    try:
        violation = row.find_elements_by_tag_name("td")[2]
        print(violation.text.strip())
        print('-----')
    except:
        pass

Respondent failed properly clean and sanitize the metal implements used at the Salon; Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution.
-----
Respondent failed to clean and sanitize four (4) whirlpool foot spas as required at the end of each day, constituting two (2) violations; Respondent failed to keep a record of the date and time of four (4) foot spas daily or bi-weekly cleaning and if the foot spas were not used, constituting two (2) violations.
-----
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
-----
Respondent failed to clean and disinfect all wax pots; Respondent failed to properly clean multi-use items prior to each service.
-----
Respondent engaged in fraud or deceit in obtaining a certificate, license, or permit.
-----
Respondent failed properly clean and sanitize the metal implements used at the Salon; Respondent failed to wipe clean and disinfect electrical e

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [7]:
for row in rows[:10]:
    try:
        complaint_number = row.find_elements_by_class_name("results_text")[5]
        print(complaint_number.text.strip())
        print('-----')
    except:
        pass

COS20190010072
-----
COS20190016762
-----
COS20200010387
-----
COS20200010502
-----
COS20190008104
-----
COS20200010511
-----
COS20200004202
-----
COS20190004199
-----
COS20200000101
-----


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [8]:
nguyen_list = []

for row in rows[:10]:
    try:
        info_dict = {}
        
        first_name = row.find_element_by_class_name("results_text")
        info_dict["nguyen_name"] = first_name.text.strip()
        #print(first_name.text.strip())
        
        violation = row.find_elements_by_tag_name("td")[2]
        info_dict["violation"] = violation.text.strip()
        #print("Violation: "+violation.text.strip())
        
        complaint_number = row.find_elements_by_class_name("results_text")[5]
        info_dict["complaint_number"] = complaint_number.text.strip()
        #print("Complaint Number: "+complaint_number.text.strip())
        
        license_number = row.find_elements_by_class_name("results_text")[4]
        info_dict["license_number"] = license_number.text.strip()
        #print("License Number: "+license_number.text.strip())
        
        zip_code = row.find_elements_by_class_name("results_text")[3]
        info_dict["zip_code"] = zip_code.text.strip()
        #print("Zip Code: "+zip_code.text.strip())
        
        county = row.find_elements_by_class_name("results_text")[2]
        info_dict["county"] = county.text.strip()
        #print("County: "+county.text.strip())
        
        city = row.find_elements_by_class_name("results_text")[1]
        info_dict["city"] = city.text.strip()
        #print("City: "+city.text.strip())
        #print('-----')
        nguyen_list.append(info_dict)
    except:
        pass
    
print(nguyen_list)

[{'nguyen_name': 'NGUYEN, MIMI PHAM', 'violation': 'Respondent failed properly clean and sanitize the metal implements used at the Salon; Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution.', 'complaint_number': 'COS20190010072', 'license_number': '784210', 'zip_code': '77449', 'county': 'HARRIS', 'city': 'KATY'}, {'nguyen_name': 'NGUYEN, HA', 'violation': 'Respondent failed to clean and sanitize four (4) whirlpool foot spas as required at the end of each day, constituting two (2) violations; Respondent failed to keep a record of the date and time of four (4) foot spas daily or bi-weekly cleaning and if the foot spas were not used, constituting two (2) violations.', 'complaint_number': 'COS20190016762', 'license_number': '764888', 'zip_code': '76017', 'county': 'TARRANT', 'city': 'ARLINGTON'}, {'nguyen_name': 'NGUYEN, THAO HONG', 'violation': 'Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements a

### Save that to a CSV

- Tip: Use `pd.DataFrame` to create a dataframe, and then save it to a CSV.

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame(nguyen_list)
df.to_csv('nguyen.zip', index=False)




### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [11]:
df_created = pd.read_csv('nguyen.zip')

In [12]:
df_created.head()

,nguyen_name,violation,complaint_number,license_number,zip_code,county,city
0,"NGUYEN, MIMI PHAM",Respondent failed properly clean and sanitize ...,COS20190010072,784210,77449,HARRIS,KATY
1,"NGUYEN, HA",Respondent failed to clean and sanitize four (...,COS20190016762,764888,76017,TARRANT,ARLINGTON
2,"NGUYEN, THAO HONG","Respondent failed to clean, disinfect, and ste...",COS20200010387,"799926, 1753491",78238,BEXAR,SAN ANTONIO
3,"NGUYEN, CINDY",Respondent failed to clean and disinfect all w...,COS20200010502,"806232, 1260359, 1280071",78414,NUECES,CORPUS CHRISTI
4,"NGUYEN, CHAU KHANH LINH",Respondent engaged in fraud or deceit in obtai...,COS20190008104,1764073,36116,OUT OF STATE,MONTGOMERY


## Let's do this an easier way

Use Selenium and `pd.read_html` to get the table as a dataframe.

In [42]:
table = driver.find_element_by_tag_name("table")
df = pd.read_html(table.get_attribute("outerHTML"))
df = df[0]
df.head()

,Name and Location,Order,Basis for Order
0,"NGUYEN, MIMI PHAM City: KATY County: HARRIS Zi...",Date: 11/12/2020Respondent is assessed an admi...,Respondent failed properly clean and sanitize ...
1,"NGUYEN, HA City: ARLINGTON County: TARRANT Zip...",Date: 11/12/2020Respondent is assessed an admi...,Respondent failed to clean and sanitize four (...
2,"NGUYEN, THAO HONG City: SAN ANTONIO County: BE...",Date: 11/12/2020Respondent is assessed an admi...,"Respondent failed to clean, disinfect, and ste..."
3,"NGUYEN, CINDY City: CORPUS CHRISTI County: NUE...",Date: 10/29/2020Respondent is assessed an admi...,Respondent failed to clean and disinfect all w...
4,"NGUYEN, CHAU KHANH LINH City: MONTGOMERY Count...",Date: 10/26/2020The Respondent's Cosmetology M...,Respondent engaged in fraud or deceit in obtai...
